In [4]:
%matplotlib inline
import pandas as pd 
import numpy as np
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
from statsmodels.formula.api import ols
from scipy import stats

In [5]:
data = pd.read_csv("playlists.csv", sep=";", encoding = "ISO-8859-1") 

### Negatives and positives

In [6]:
companies = data['company'].unique()
by_company = [data[data.company == company] for company in companies]
positives = []
negatives = []
for data_com in by_company:
    data_com = data_com.sort_values('playlist_sample')
    last_pl = int(data_com.tail(1).playlist_sample)
    #pls = pd.DataFrame({'pl':range (1,last_pl+1), 'old':[0]*last_pl, 'new':[0]*last_pl})
    #curr_pl = data_com.query('playlist_sample == '+str(1))
    #pls.new[0]=(curr_pl.shape[0])/3
    #sum_pls = [curr_pl.shape[0]/3]
    #for i in range(2,last_pl+1):
    #    curr_pl = data_com.query('playlist_sample == '+str(i))
    #    pre_pl = data_com.query('playlist_sample == '+str(i-1))
    #    olds = curr_pl['song'].map(pre_pl['song'].value_counts()).sum(axis = 0)/3
    #    pls.old[i-1]= olds/3 
    #    pls.new[i-1]=(curr_pl.shape[0]-olds)/3
    #    sum_pls.append(curr_pl.shape[0]/3)
    #sum_pls = pd.DataFrame(sum_pls)
    #pls_rel = (pls[['old']].div(sum_pls.values, axis=0)*100).join(pls[['new']].div(sum_pls.values, axis=0)*100)
    #pls[['old','new']].plot(kind='bar', stacked=True, title=data_com.iloc[0,0])
    #for n in pls_rel:
    #    for i, (cs, ab, pc) in enumerate(zip(pls.iloc[:, 1:].cumsum(1)[n].values, pls[n].values, pls_rel[n].values)):
    #        if(pc>0):
    #            plt.text(i, cs - ab/2, str(np.round(pc, 1)) + '%', va='center', ha='center')
    df_last_pl= data_com.query('playlist_sample == '+str(last_pl))
    positives.append(df_last_pl)
    pos_loc = pd.DataFrame({}, columns=data_com.columns)
    rep=0
    for index, row in data_com[data_com.playlist_sample<last_pl].iterrows(): 
        if not ((df_last_pl['artist'] == row['artist']) & (df_last_pl['song'] ==  row['song'])).any() and pos_loc[(pos_loc['artist']== row['artist']) & (pos_loc['song'] ==  row['song'])].shape[0]<3:
            pos_loc= pos_loc.append(row, ignore_index=True)
        else:
            rep+=1
    #n_vs_p = pd.DataFrame({'sam':['pos', 'neg'],'num':[df_last_pl.shape[0]/3,pos_loc.shape[0]/3]})
    # n_vs_p.plot.bar(x='sam', y='num', rot=0, title=data_com.iloc[0,0])
    negatives.append(pos_loc)

Count of positives negatives and total by company.

In [7]:
for i in range(len(companies)):
    print(companies[i])
    print("Negatives: ",negatives[i].shape[0]/3)
    print("Positives: ",positives[i].shape[0]/3)
    print("Total: ", by_company[i].shape[0]/3)

Arte Francés
Negatives:  76.0
Positives:  29.0
Total:  147.0
Club De Banqueros y Empresarios
Negatives:  61.0
Positives:  24.0
Total:  88.0
Gramma
Negatives:  51.0
Positives:  21.0
Total:  103.0
Hotel Marrakech
Negatives:  28.0
Positives:  20.0
Total:  68.0
Specialized
Negatives:  38.0
Positives:  35.0
Total:  103.0
Urban Place
Negatives:  48.0
Positives:  24.0
Total:  85.0


Append the positives and negatives records with a new column "chosen" that takes value of 1 if is chosen.

In [8]:
df_n_ps = []
for i in range(len(negatives)):
    negatives[i]['chosen']=0
    positives[i]['chosen']=1
    df_n_ps.append(negatives[i].append(positives[i]))

D:\Usuarios\1144084318\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


### Preprocessing

In [9]:
from collections import Counter


from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, accuracy_score, silhouette_samples, silhouette_score, calinski_harabaz_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
import math
import seaborn as sns

In [10]:
for i in range(len(companies)):
    df_n_ps[i].bitratefiles = df_n_ps[i].bitratefiles.astype('float64')
    df_n_ps[i].pitchfiles = df_n_ps[i].pitchfiles.astype('float64')
    df_n_ps[i].bestkeyfiles = df_n_ps[i].bestkeyfiles.astype('float64')
df_n_ps[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 315 entries, 0 to 179
Data columns (total 65 columns):
company                 315 non-null object
playlist_sample         315 non-null object
namesfiles              315 non-null object
no                      315 non-null object
artist                  315 non-null object
song                    315 non-null object
sampleratefiles         315 non-null object
totalsamplesfiles       315 non-null object
durationfiles           315 non-null float64
bitratefiles            315 non-null float64
rmsfiles                315 non-null float64
rmsmedianfiles          315 non-null float64
lowenergyfiles          315 non-null float64
ASRfiles                315 non-null float64
beatspectrumfiles       315 non-null float64
eventdensityfiles       315 non-null float64
tempofiles              315 non-null float64
pulseclarityfiles       315 non-null float64
zerocrossfiles          315 non-null float64
rolloffsfiles           315 non-null float64
bri

Vamos a reemplazar los NaN y entonces a normalizar los datos para que todas las variables tengan la misma importancia. Solo vamos a considerar los datos numéricos.

In [11]:
df_n_ps_std = [0]*len(companies)
for i in range(len(companies)):
    df_n_ps[i] = df_n_ps[i].fillna(0)
    df_n_ps_std[i] = pd.DataFrame(preprocessing.scale(df_n_ps[i].iloc[:,8:]))
    df_n_ps_std[i].columns=df_n_ps[i].columns[8:]
df_n_ps_std[0].mean(axis=0)

durationfiles          -1.889141e-16
bitratefiles            0.000000e+00
rmsfiles                3.559763e-16
rmsmedianfiles         -2.396672e-16
lowenergyfiles          1.543739e-16
ASRfiles                1.226532e-16
beatspectrumfiles       2.269789e-16
eventdensityfiles      -6.132661e-17
tempofiles              4.103860e-16
pulseclarityfiles      -6.696583e-17
zerocrossfiles         -1.092600e-16
rolloffsfiles           2.661011e-16
brightnessfiles         1.092600e-16
spreadfiles             1.519067e-16
centroidfiles           1.501444e-16
kurtosisfiles           1.875043e-16
flatnessfiles          -4.017950e-17
entropyfiles            6.012827e-16
mfccfiles_1            -4.398598e-16
mfccfiles_2            -2.326182e-17
mfccfiles_3             6.308886e-17
mfccfiles_4             1.718202e-17
mfccfiles_5             1.832749e-17
mfccfiles_6            -3.172066e-17
mfccfiles_7            -1.400996e-16
mfccfiles_8             5.110550e-17
mfccfiles_9             3.101575e-17
m

In [12]:
df_n_ps_std[0].std(axis=0)

durationfiles           1.001591
bitratefiles            0.000000
rmsfiles                1.001591
rmsmedianfiles          1.001591
lowenergyfiles          1.001591
ASRfiles                1.001591
beatspectrumfiles       1.001591
eventdensityfiles       1.001591
tempofiles              1.001591
pulseclarityfiles       1.001591
zerocrossfiles          1.001591
rolloffsfiles           1.001591
brightnessfiles         1.001591
spreadfiles             1.001591
centroidfiles           1.001591
kurtosisfiles           1.001591
flatnessfiles           1.001591
entropyfiles            1.001591
mfccfiles_1             1.001591
mfccfiles_2             1.001591
mfccfiles_3             1.001591
mfccfiles_4             1.001591
mfccfiles_5             1.001591
mfccfiles_6             1.001591
mfccfiles_7             1.001591
mfccfiles_8             1.001591
mfccfiles_9             1.001591
mfccfiles_10            1.001591
mfccfiles_11            1.001591
mfccfiles_12            1.001591
mfccfiles_

Borramos pitch y bitrate porque todos sus valores son 0.

In [13]:
for i in range(len(companies)):
    df_n_ps_std[i] = df_n_ps_std[i].drop(columns="pitchfiles")
    df_n_ps_std[i] = df_n_ps_std[i].drop(columns="bitratefiles")

In [14]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.0'

In [15]:
from keras.layers import Input, Flatten, Dense#, Lambda
from keras.models import Model
from keras import layers
from keras import models, optimizers

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV #permite buscar la mejor configuración de parámetros con C-V
from sklearn.metrics import make_scorer # permite crear una clase scorer a partir de una función de score (necesario para el kappa)
from sklearn.metrics import accuracy_score, cohen_kappa_score, classification_report, roc_auc_score
from sklearn.model_selection import train_test_split #metodo de particionamiento de datasets para evaluación
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

# MFCC

In [16]:
df_n_ps_std[0].columns[17:30]

Index(['mfccfiles_1', 'mfccfiles_2', 'mfccfiles_3', 'mfccfiles_4',
       'mfccfiles_5', 'mfccfiles_6', 'mfccfiles_7', 'mfccfiles_8',
       'mfccfiles_9', 'mfccfiles_10', 'mfccfiles_11', 'mfccfiles_12',
       'mfccfiles_13'],
      dtype='object')

In [17]:
df_n_ps_std_mfcc = [None]*len(companies)
for i in range(len(companies)):
    df_n_ps_std_mfcc[i] = pd.DataFrame(df_n_ps_std[i].iloc[:,17:30])
    df_n_ps_std_mfcc[i].columns=df_n_ps_std[i].columns[17:30]
df_n_ps_std_mfcc[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 13 columns):
mfccfiles_1     315 non-null float64
mfccfiles_2     315 non-null float64
mfccfiles_3     315 non-null float64
mfccfiles_4     315 non-null float64
mfccfiles_5     315 non-null float64
mfccfiles_6     315 non-null float64
mfccfiles_7     315 non-null float64
mfccfiles_8     315 non-null float64
mfccfiles_9     315 non-null float64
mfccfiles_10    315 non-null float64
mfccfiles_11    315 non-null float64
mfccfiles_12    315 non-null float64
mfccfiles_13    315 non-null float64
dtypes: float64(13)
memory usage: 32.1 KB


## 2-comp groups combinations

In [ ]:
two_combs = list(itertools.combinations(range(len(companies)), 2))
for comb in two_combs:
    print('## '+companies[comb[0]].upper()+' y '+companies[comb[1]].upper())
    X = df_n_ps_std_mfcc[comb[0]].append(df_n_ps_std_mfcc[comb[1]])
    y = df_n_ps[comb[0]]['chosen'].append(df_n_ps[comb[1]]['chosen'])

    print(X.head())
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))

    activation_vec = ['logistic', 'relu', 'tanh']
    max_iter_vec = [10, 20, 50, 75, 100, 200, 300, 400, 500, 1000, 2000]
    hidden_layer_sizes_vec = [(10,), (20,), (30,), (10, 10), (20, 20), (30, 30), (20, 10), 
                              (10, 10, 10), (20, 20, 20), (30, 30, 30), (30, 20, 10)]
    learning_rate_init_vec = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02]
    batch_size_vec = [10, 20, 40, 60, 80, 100, 150]


    np.random.seed(1234)
    parametros = {'activation': activation_vec,
                  'max_iter':max_iter_vec,
                  'hidden_layer_sizes': hidden_layer_sizes_vec,
                  'learning_rate_init': learning_rate_init_vec#,
                  #'batch_size': batch_size_vec
                  }
    scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
    grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1, iid=True)

    grid.fit(X_train, y_train)

    print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}".format(
        grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))

    n0=X_train.shape[1]
    ### hidden_layer_sizes
    ns = []
    for i in range (len(grid.best_params_['hidden_layer_sizes'])):
        ns.append(grid.best_params_['hidden_layer_sizes'][i])

    ns.append(1)
    lr = grid.best_params_['learning_rate_init']
    epochs = grid.best_params_['max_iter']
    if grid.best_params_['activation'] == 'logistic':
        grid.best_params_['activation'] = 'sigmoid'
    
    input_tensor = Input(shape = (n0,))
    hidden_outputs = [input_tensor]
    for i in range (len(ns)-1):
        hidden_outputs.append(Dense(ns[i], activation = grid.best_params_['activation'])(hidden_outputs[i]))
    classification_output = Dense(ns[-1], activation = 'sigmoid')(hidden_outputs[-1])

    model = Model([input_tensor], [classification_output])
    weights = model.get_weights()

    print(model.summary())

    model.set_weights(weights)
    adam = keras.optimizers.Adam(lr=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=0, 
                callbacks=[
                    keras.callbacks.ReduceLROnPlateau(
                        monitor='val_accuracy', factor=0.5, patience=10, min_delta=0.01, verbose=0
                    )
                ]
             )

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    print("epochs: "+str(len(acc)))

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print("test loss: {}, test accuracy: {}".format(test_loss, test_acc))

    y_pred = model.predict(X_test)
    print("AUC ROC: ",roc_auc_score(y_test, y_pred))

    y_pred = list(map(lambda i: int(i>=0.5), y_pred))
    print("Kappa: ",cohen_kappa_score(y_test, y_pred))

    print(confusion_matrix(y_test, y_pred))


In [ ]:
from IPython.display import display, Markdown, Latex
two_combs = list(itertools.combinations(range(len(companies)), 2))
for comb in two_combs[2:]:
    display(Markdown('## '+companies[comb[0]]+' y '+companies[comb[1]]))
    X = df_n_ps_std_mfcc[comb[0]].append(df_n_ps_std_mfcc[comb[1]])
    y = df_n_ps[comb[0]]['chosen'].append(df_n_ps[comb[1]]['chosen'])

    print(X.head())
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))

    activation_vec = ['logistic', 'relu', 'tanh']
    max_iter_vec = [10, 20, 50, 75, 100, 200, 300, 400, 500, 1000, 2000]
    hidden_layer_sizes_vec = [(10,), (20,), (30,), (10, 10), (20, 20), (30, 30), (20, 10), 
                              (10, 10, 10), (20, 20, 20), (30, 30, 30), (30, 20, 10)]
    learning_rate_init_vec = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02]
    batch_size_vec = [10, 20, 40, 60, 80, 100, 150]


    np.random.seed(1234)
    parametros = {'activation': activation_vec,
                  'max_iter':max_iter_vec,
                  'hidden_layer_sizes': hidden_layer_sizes_vec,
                  'learning_rate_init': learning_rate_init_vec#,
                  #'batch_size': batch_size_vec
                  }
    scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
    grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1, iid=True)

    grid.fit(X_train, y_train)

    print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}".format(
        grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))

    n0=X_train.shape[1]
    ### hidden_layer_sizes
    ns = []
    for i in range (len(grid.best_params_['hidden_layer_sizes'])):
        ns.append(grid.best_params_['hidden_layer_sizes'][i])

    ns.append(1)
    lr = grid.best_params_['learning_rate_init']
    epochs = grid.best_params_['max_iter']

    input_tensor = Input(shape = (n0,))
    hidden_outputs = [input_tensor]
    if grid.best_params_['activation'] == 'logistic':
        grid.best_params_['activation'] = 'sigmoid'
    for i in range (len(ns)-1):
        hidden_outputs.append(Dense(ns[i], activation = grid.best_params_['activation'])(hidden_outputs[i]))
    classification_output = Dense(ns[-1], activation = 'sigmoid')(hidden_outputs[-1])

    model = Model([input_tensor], [classification_output])
    weights = model.get_weights()

    print(model.summary())

    model.set_weights(weights)
    adam = keras.optimizers.Adam(lr=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=0, 
                callbacks=[
                    keras.callbacks.ReduceLROnPlateau(
                        monitor='val_accuracy', factor=0.5, patience=10, min_delta=0.01, verbose=0
                    )
                ]
             )

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    print("epochs: "+str(len(acc)))

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print("test loss: {}, test accuracy: {}".format(test_loss, test_acc))

    y_pred = model.predict(X_test)
    print("AUC ROC: ",roc_auc_score(y_test, y_pred))

    y_pred = list(map(lambda i: int(i>=0.5), y_pred))
    print("Kappa: ",cohen_kappa_score(y_test, y_pred))

    print(confusion_matrix(y_test, y_pred))


In [3]:
from IPython.display import display, Markdown, Latex
two_combs = list(itertools.combinations(range(len(companies)), 2))
for comb in two_combs[4:]:
    display(Markdown('## '+companies[comb[0]]+' y '+companies[comb[1]]))
    X = df_n_ps_std_mfcc[comb[0]].append(df_n_ps_std_mfcc[comb[1]])
    y = df_n_ps[comb[0]]['chosen'].append(df_n_ps[comb[1]]['chosen'])

    print(X.head())
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))

    activation_vec = ['logistic', 'relu', 'tanh']
    max_iter_vec = [10, 20, 50, 75, 100, 200, 300, 400, 500, 1000, 2000]
    hidden_layer_sizes_vec = [(10,), (20,), (30,), (10, 10), (20, 20), (30, 30), (20, 10), 
                              (10, 10, 10), (20, 20, 20), (30, 30, 30), (30, 20, 10)]
    learning_rate_init_vec = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02]
    batch_size_vec = [10, 20, 40, 60, 80, 100, 150]


    np.random.seed(1234)
    parametros = {'activation': activation_vec,
                  'max_iter':max_iter_vec,
                  'hidden_layer_sizes': hidden_layer_sizes_vec,
                  'learning_rate_init': learning_rate_init_vec#,
                  #'batch_size': batch_size_vec
                  }
    scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
    grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1, iid=True)

    grid.fit(X_train, y_train)

    print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}".format(
        grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))

    n0=X_train.shape[1]
    ### hidden_layer_sizes
    ns = []
    for i in range (len(grid.best_params_['hidden_layer_sizes'])):
        ns.append(grid.best_params_['hidden_layer_sizes'][i])

    ns.append(1)
    lr = grid.best_params_['learning_rate_init']
    epochs = grid.best_params_['max_iter']

    input_tensor = Input(shape = (n0,))
    hidden_outputs = [input_tensor]
    if grid.best_params_['activation'] == 'logistic':
        grid.best_params_['activation'] = 'sigmoid'
    for i in range (len(ns)-1):
        hidden_outputs.append(Dense(ns[i], activation = grid.best_params_['activation'])(hidden_outputs[i]))
    classification_output = Dense(ns[-1], activation = 'sigmoid')(hidden_outputs[-1])

    model = Model([input_tensor], [classification_output])
    weights = model.get_weights()

    print(model.summary())

    model.set_weights(weights)
    adam = keras.optimizers.Adam(lr=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=0, 
                callbacks=[
                    keras.callbacks.ReduceLROnPlateau(
                        monitor='val_accuracy', factor=0.5, patience=10, min_delta=0.01, verbose=0
                    )
                ]
             )

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    print("epochs: "+str(len(acc)))

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print("test loss: {}, test accuracy: {}".format(test_loss, test_acc))

    y_pred = model.predict(X_test)
    print("AUC ROC: ",roc_auc_score(y_test, y_pred))

    y_pred = list(map(lambda i: int(i>=0.5), y_pred))
    print("Kappa: ",cohen_kappa_score(y_test, y_pred))

    print(confusion_matrix(y_test, y_pred))


NameError: name 'companies' is not defined

## 3-comp groups combinations

In [18]:
from IPython.display import display, Markdown, Latex
import itertools
three_combs = list(itertools.combinations(range(len(companies)), 3))
for comb in three_combs:
    title = '## '
    for no in comb[:-1]:
        title+=companies[no]
        title+=' y '
    title+=companies[comb[-1]]
    display(Markdown(title))
    X = df_n_ps_std_mfcc[comb[0]]
    y = df_n_ps[comb[0]]['chosen']
    for no in comb[1:]:
        X= X.append(df_n_ps_std_mfcc[no])
        y= y.append(df_n_ps[no]['chosen'])
    

    print(X.head())
    print(X.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))

    activation_vec = ['logistic', 'relu', 'tanh']
    max_iter_vec = [10, 20, 50, 75, 100, 200, 300, 400, 500, 1000, 2000]
    hidden_layer_sizes_vec = [(10,), (20,), (30,), (10, 10), (20, 20), (30, 30), (20, 10), 
                              (10, 10, 10), (20, 20, 20), (30, 30, 30), (30, 20, 10)]
    learning_rate_init_vec = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02]
    batch_size_vec = [10, 20, 40, 60, 80, 100, 150]


    np.random.seed(1234)
    parametros = {'activation': activation_vec,
                  'max_iter':max_iter_vec,
                  'hidden_layer_sizes': hidden_layer_sizes_vec,
                  'learning_rate_init': learning_rate_init_vec#,
                  #'batch_size': batch_size_vec
                  }
    scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
    grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1, iid=True)

    grid.fit(X_train, y_train)

    print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}".format(
        grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))

    n0=X_train.shape[1]
    ### hidden_layer_sizes
    ns = []
    for i in range (len(grid.best_params_['hidden_layer_sizes'])):
        ns.append(grid.best_params_['hidden_layer_sizes'][i])

    ns.append(1)
    lr = grid.best_params_['learning_rate_init']
    epochs = grid.best_params_['max_iter']

    input_tensor = Input(shape = (n0,))
    hidden_outputs = [input_tensor]
    if grid.best_params_['activation'] == 'logistic':
        grid.best_params_['activation'] = 'sigmoid'
    for i in range (len(ns)-1):
        hidden_outputs.append(Dense(ns[i], activation = grid.best_params_['activation'])(hidden_outputs[i]))
    classification_output = Dense(ns[-1], activation = 'sigmoid')(hidden_outputs[-1])

    model = Model([input_tensor], [classification_output])
    weights = model.get_weights()

    print(model.summary())

    model.set_weights(weights)
    adam = keras.optimizers.Adam(lr=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=0, 
                callbacks=[
                    keras.callbacks.ReduceLROnPlateau(
                        monitor='val_accuracy', factor=0.5, patience=10, min_delta=0.01, verbose=0
                    )
                ]
             )

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    print("epochs: "+str(len(acc)))

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print("test loss: {}, test accuracy: {}".format(test_loss, test_acc))

    y_pred = model.predict(X_test)
    print("AUC ROC: ",roc_auc_score(y_test, y_pred))

    y_pred = list(map(lambda i: int(i>=0.5), y_pred))
    print("Kappa: ",cohen_kappa_score(y_test, y_pred))

    print(confusion_matrix(y_test, y_pred))

## Arte Francés y Club De Banqueros y Empresarios y Gramma

   mfccfiles_1  mfccfiles_2  mfccfiles_3  mfccfiles_4  mfccfiles_5  \
0     0.297583     1.225637    -0.367641     0.606499     0.072373   
1     0.637676    -1.507256    -1.572737    -0.954161    -0.857425   
2     2.236730    -0.319414     0.669910    -1.918119    -0.820882   
3     0.662077    -0.381499     0.111981    -1.743808    -1.317593   
4     0.736502     0.112932    -0.065024    -1.049458    -0.408043   

   mfccfiles_6  mfccfiles_7  mfccfiles_8  mfccfiles_9  mfccfiles_10  \
0    -2.029620     0.791469     0.752018     2.268802     -1.383289   
1     0.327005     0.816764     0.214245     0.241703      0.637066   
2    -2.379333    -1.570021    -2.755344    -2.150610     -2.528577   
3    -1.348534    -0.627198    -1.629882    -2.075974     -1.248765   
4    -0.437499     0.090831    -0.852983    -1.922491     -0.284365   

   mfccfiles_11  mfccfiles_12  mfccfiles_13  
0      0.548279      1.903211     -1.011470  
1      1.601538      0.300317     -0.466779  
2     -0.87708

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Los parámetros del mejor modelo fueron {'activation': 'tanh', 'hidden_layer_sizes': (30, 30, 30), 'learning_rate_init': 0.001, 'max_iter': 100}, que permiten obtener un Accuracy de 74.70% y un Kappa del 26.53
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 13)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)                420       
_________________________________________________________________
dense_2 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_3 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 31        
Total params: 2,311
Trainable params: 2,311
Non-

## Arte Francés y Club De Banqueros y Empresarios y Hotel Marrakech

...

   mfccfiles_1  mfccfiles_2  mfccfiles_3  mfccfiles_4  mfccfiles_5  \
0    -0.674917     0.169246     0.673543     1.157142    -0.633186   
1     0.277269     0.514176     0.200398     0.988939    -1.756594   
2     1.483921     0.724793     0.473099     0.439577    -0.358096   
3    -0.734008    -0.683844    -0.764866    -0.225060    -0.261235   
4    -0.834815    -0.735908    -1.177596    -0.093532     0.508050   

   mfccfiles_6  mfccfiles_7  mfccfiles_8  mfccfiles_9  mfccfiles_10  \
0     0.688145     0.215883    -0.452048     1.101066      0.064017   
1    -0.022788    -0.235704     0.523508    -0.604231      1.188209   
2    -0.452581    -0.213173    -0.596057    -0.767473      0.696227   
3    -0.243429     0.588768     0.874148     1.302526      0.091256   
4     0.503458     1.380798     1.847226     1.227896      0.017729   

   mfccfiles_11  mfccfiles_12  mfccfiles_13  
0     -0.153703      1.751289      0.812723  
1      0.863617     -0.801768      0.229305  
2     -0.11125

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Los parámetros del mejor modelo fueron {'activation': 'tanh', 'hidden_layer_sizes': (30,), 'learning_rate_init': 0.002, 'max_iter': 300}, que permiten obtener un Accuracy de 68.44% y un Kappa del 27.07
Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 13)                0         
_________________________________________________________________
dense_52 (Dense)             (None, 30)                420       
_________________________________________________________________
dense_53 (Dense)             (None, 1)                 31        
Total params: 451
Trainable params: 451
Non-trainable params: 0
_________________________________________________________________
None
epochs: 300
163/163 [==============================] - 0s 43us/step
test loss: 0.6811560271707781, test accuracy: 0.5828220844268799
AUC ROC:  0.5981424148606811
Kappa:  0.03751302535

## Hotel Marrakech y Specialized y Urban Place

   mfccfiles_1  mfccfiles_2  mfccfiles_3  mfccfiles_4  mfccfiles_5  \
0     0.221235     1.617887     0.929874    -0.231486    -0.525862   
1     0.836735    -0.529605    -1.268139    -0.791053     0.815880   
2    -0.190995     1.202756     0.050028    -2.631154     3.701544   
3     0.521202     1.354284     1.423683    -0.634173     0.934734   
4     0.250234     1.586078    -1.791096     0.127156     1.573000   

   mfccfiles_6  mfccfiles_7  mfccfiles_8  mfccfiles_9  mfccfiles_10  \
0     1.384826     0.709441     0.512679    -2.231286     -2.278872   
1    -1.992230    -0.371430    -0.356669     1.323871      0.946394   
2    -1.158173     0.439586     2.317548    -2.282526     -1.571775   
3     0.214772    -0.349135     1.009101    -2.193012     -0.301254   
4     0.288525     1.962471     1.500627     1.352853     -1.921935   

   mfccfiles_11  mfccfiles_12  mfccfiles_13  
0     -0.728806     -2.187766     -1.206544  
1     -1.085097      0.673490     -1.496313  
2     -2.54195

# Tonal Centroid

In [19]:
df_n_ps_std[0].columns

Index(['durationfiles', 'rmsfiles', 'rmsmedianfiles', 'lowenergyfiles',
       'ASRfiles', 'beatspectrumfiles', 'eventdensityfiles', 'tempofiles',
       'pulseclarityfiles', 'zerocrossfiles', 'rolloffsfiles',
       'brightnessfiles', 'spreadfiles', 'centroidfiles', 'kurtosisfiles',
       'flatnessfiles', 'entropyfiles', 'mfccfiles_1', 'mfccfiles_2',
       'mfccfiles_3', 'mfccfiles_4', 'mfccfiles_5', 'mfccfiles_6',
       'mfccfiles_7', 'mfccfiles_8', 'mfccfiles_9', 'mfccfiles_10',
       'mfccfiles_11', 'mfccfiles_12', 'mfccfiles_13', 'inharmonicityfiles',
       'bestkeyfiles', 'keyclarityfiles', 'modalityfiles',
       'tonalcentroidfiles_1', 'tonalcentroidfiles_2', 'tonalcentroidfiles_3',
       'tonalcentroidfiles_4', 'tonalcentroidfiles_5', 'tonalcentroidfiles_6',
       'chromagramfiles_1', 'chromagramfiles_2', 'chromagramfiles_3',
       'chromagramfiles_4', 'chromagramfiles_5', 'chromagramfiles_6',
       'chromagramfiles_7', 'chromagramfiles_8', 'chromagramfiles_9',
      

In [20]:
df_n_ps_std[0].columns[34:40]

Index(['tonalcentroidfiles_1', 'tonalcentroidfiles_2', 'tonalcentroidfiles_3',
       'tonalcentroidfiles_4', 'tonalcentroidfiles_5', 'tonalcentroidfiles_6'],
      dtype='object')

In [21]:
df_n_ps_std_tc = [None]*len(companies)
for i in range(len(companies)):
    df_n_ps_std_tc[i] = pd.DataFrame(df_n_ps_std[i].iloc[:,34:40])
    df_n_ps_std_tc[i].columns=df_n_ps_std[i].columns[34:40]
df_n_ps_std_tc[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 6 columns):
tonalcentroidfiles_1    315 non-null float64
tonalcentroidfiles_2    315 non-null float64
tonalcentroidfiles_3    315 non-null float64
tonalcentroidfiles_4    315 non-null float64
tonalcentroidfiles_5    315 non-null float64
tonalcentroidfiles_6    315 non-null float64
dtypes: float64(6)
memory usage: 14.8 KB


## 2-comp groups combinations

In [28]:
from IPython.display import display, Markdown, Latex
import itertools
two_combs = list(itertools.combinations(range(len(companies)), 2))
for comb in two_combs:
    display(Markdown('## '+companies[comb[0]]+' y '+companies[comb[1]]))
    X = df_n_ps_std_tc[comb[0]].append(df_n_ps_std_tc[comb[1]])
    y = df_n_ps[comb[0]]['chosen'].append(df_n_ps[comb[1]]['chosen'])

    print(X.head())
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))

    activation_vec = ['logistic', 'relu', 'tanh']
    max_iter_vec = [10, 20, 50, 75, 100, 200, 300, 400, 500, 1000, 2000]
    hidden_layer_sizes_vec = [(10,), (20,), (30,), (10, 10), (20, 20), (30, 30), (20, 10), 
                              (10, 10, 10), (20, 20, 20), (30, 30, 30), (30, 20, 10)]
    learning_rate_init_vec = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02]
    batch_size_vec = [10, 20, 40, 60, 80, 100, 150]


    np.random.seed(1234)
    parametros = {'activation': activation_vec,
                  'max_iter':max_iter_vec,
                  'hidden_layer_sizes': hidden_layer_sizes_vec,
                  'learning_rate_init': learning_rate_init_vec#,
                  #'batch_size': batch_size_vec
                  }
    scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
    grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1, iid=True)

    grid.fit(X_train, y_train)

    print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}".format(
        grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))

    n0=X_train.shape[1]
    ### hidden_layer_sizes
    ns = []
    for i in range (len(grid.best_params_['hidden_layer_sizes'])):
        ns.append(grid.best_params_['hidden_layer_sizes'][i])

    ns.append(1)
    lr = grid.best_params_['learning_rate_init']
    epochs = grid.best_params_['max_iter']

    input_tensor = Input(shape = (n0,))
    hidden_outputs = [input_tensor]
    if grid.best_params_['activation'] == 'logistic':
        grid.best_params_['activation'] = 'sigmoid'
    for i in range (len(ns)-1):
        hidden_outputs.append(Dense(ns[i], activation = grid.best_params_['activation'])(hidden_outputs[i]))
    classification_output = Dense(ns[-1], activation = 'sigmoid')(hidden_outputs[-1])

    model = Model([input_tensor], [classification_output])
    weights = model.get_weights()

    print(model.summary())

    model.set_weights(weights)
    adam = keras.optimizers.Adam(lr=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=0, 
                callbacks=[
                    keras.callbacks.ReduceLROnPlateau(
                        monitor='val_accuracy', factor=0.5, patience=10, min_delta=0.01, verbose=0
                    )
                ]
             )

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    print("epochs: "+str(len(acc)))

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print("test loss: {}, test accuracy: {}".format(test_loss, test_acc))

    y_pred = model.predict(X_test)
    print("AUC ROC: ",roc_auc_score(y_test, y_pred))

    y_pred = list(map(lambda i: int(i>=0.5), y_pred))
    print("Kappa: ",cohen_kappa_score(y_test, y_pred))

    print(confusion_matrix(y_test, y_pred))

## Arte Francés y Club De Banqueros y Empresarios

   tonalcentroidfiles_1  tonalcentroidfiles_2  tonalcentroidfiles_3  \
0              1.585484              0.923309             -0.748807   
1              1.129768              0.963814              0.209096   
2             -0.066076              1.857866              1.921193   
3              0.119831              1.429286              1.472808   
4             -0.123292              0.197415              0.503797   

   tonalcentroidfiles_4  tonalcentroidfiles_5  tonalcentroidfiles_6  
0              1.209820             -1.073924              0.283035  
1             -0.143471              0.115184              0.020905  
2              1.093619              2.089353              1.984310  
3              1.282029              1.096897              2.122871  
4              1.431215              1.715761              0.683611  


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Los parámetros del mejor modelo fueron {'activation': 'tanh', 'hidden_layer_sizes': (30, 20, 10), 'learning_rate_init': 0.009, 'max_iter': 20}, que permiten obtener un Accuracy de 74.24% y un Kappa del 11.33
Model: "model_76"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_76 (InputLayer)        (None, 6)                 0         
_________________________________________________________________
dense_226 (Dense)            (None, 30)                210       
_________________________________________________________________
dense_227 (Dense)            (None, 20)                620       
_________________________________________________________________
dense_228 (Dense)            (None, 10)                210       
_________________________________________________________________
dense_229 (Dense)            (None, 1)                 11        
Total params: 1,051
Trainable params: 1,051
Non-

## Arte Francés y Gramma

...

Los parámetros del mejor modelo fueron {'activation': 'relu', 'hidden_layer_sizes': (30, 20, 10), 'learning_rate_init': 0.003, 'max_iter': 100}, que permiten obtener un Accuracy de 61.03% y un Kappa del 20.53
Model: "model_88"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_88 (InputLayer)        (None, 6)                 0         
_________________________________________________________________
dense_264 (Dense)            (None, 30)                210       
_________________________________________________________________
dense_265 (Dense)            (None, 20)                620       
_________________________________________________________________
dense_266 (Dense)            (None, 10)                210       
_________________________________________________________________
dense_267 (Dense)            (None, 1)                 11        
Total params: 1,051
Trainable params: 1,051
Non

## Hotel Marrakech y Urban Place

   tonalcentroidfiles_1  tonalcentroidfiles_2  tonalcentroidfiles_3  \
0              1.297447              0.231356              0.546295   
1              0.766614             -1.227377             -0.605576   
2             -0.431734             -1.183099              1.287522   
3              0.078607             -1.572049              1.188424   
4              0.007655             -1.466562              1.294323   

   tonalcentroidfiles_4  tonalcentroidfiles_5  tonalcentroidfiles_6  
0              1.007345              0.269632              0.236051  
1             -1.216250              0.473081             -0.446139  
2              0.927452             -0.309857             -1.666006  
3              0.092330             -0.244414             -0.490911  
4             -0.410616             -0.763440             -1.007284  
Los parámetros del mejor modelo fueron {'activation': 'relu', 'hidden_layer_sizes': (30, 30, 30), 'learning_rate_init': 0.004, 'max_iter': 1000}, que per

## Specialized y Urban Place

   tonalcentroidfiles_1  tonalcentroidfiles_2  tonalcentroidfiles_3  \
0             -1.035481              1.779354              1.874576   
1              0.965487             -0.399971             -1.606069   
2             -0.141249             -1.969933             -0.960470   
3             -1.590590             -0.729741             -0.575342   
4             -0.391524             -0.894181             -0.426309   

   tonalcentroidfiles_4  tonalcentroidfiles_5  tonalcentroidfiles_6  
0              0.924814             -0.129662              2.608421  
1              0.008311              0.834341              0.513694  
2              1.005123             -1.117123             -2.399517  
3              0.587988             -0.885561             -0.752828  
4              1.017585             -0.391173             -0.920259  
Los parámetros del mejor modelo fueron {'activation': 'relu', 'hidden_layer_sizes': (20, 20, 20), 'learning_rate_init': 0.007, 'max_iter': 500}, que perm

## 3-comp groups combinations

In [22]:
from IPython.display import display, Markdown, Latex
import itertools
three_combs = list(itertools.combinations(range(len(companies)), 3))
for comb in three_combs:
    title = '## '
    for no in comb[:-1]:
        title+=companies[no]
        title+=' y '
    title+=companies[comb[-1]]
    display(Markdown(title))
    X = df_n_ps_std_tc[comb[0]]
    y = df_n_ps[comb[0]]['chosen']
    for no in comb[1:]:
        X= X.append(df_n_ps_std_tc[no])
        y= y.append(df_n_ps[no]['chosen'])
    

    print(X.head())
    print(X.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))

    activation_vec = ['logistic', 'relu', 'tanh']
    max_iter_vec = [10, 20, 50, 75, 100, 200, 300, 400, 500, 1000, 2000]
    hidden_layer_sizes_vec = [(10,), (20,), (30,), (10, 10), (20, 20), (30, 30), (20, 10), 
                              (10, 10, 10), (20, 20, 20), (30, 30, 30), (30, 20, 10)]
    learning_rate_init_vec = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02]
    batch_size_vec = [10, 20, 40, 60, 80, 100, 150]


    np.random.seed(1234)
    parametros = {'activation': activation_vec,
                  'max_iter':max_iter_vec,
                  'hidden_layer_sizes': hidden_layer_sizes_vec,
                  'learning_rate_init': learning_rate_init_vec#,
                  #'batch_size': batch_size_vec
                  }
    scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
    grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1, iid=True)

    grid.fit(X_train, y_train)

    print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}".format(
        grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))

    n0=X_train.shape[1]
    ### hidden_layer_sizes
    ns = []
    for i in range (len(grid.best_params_['hidden_layer_sizes'])):
        ns.append(grid.best_params_['hidden_layer_sizes'][i])

    ns.append(1)
    lr = grid.best_params_['learning_rate_init']
    epochs = grid.best_params_['max_iter']

    input_tensor = Input(shape = (n0,))
    hidden_outputs = [input_tensor]
    if grid.best_params_['activation'] == 'logistic':
        grid.best_params_['activation'] = 'sigmoid'
    for i in range (len(ns)-1):
        hidden_outputs.append(Dense(ns[i], activation = grid.best_params_['activation'])(hidden_outputs[i]))
    classification_output = Dense(ns[-1], activation = 'sigmoid')(hidden_outputs[-1])

    model = Model([input_tensor], [classification_output])
    weights = model.get_weights()

    print(model.summary())

    model.set_weights(weights)
    adam = keras.optimizers.Adam(lr=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=0, 
                callbacks=[
                    keras.callbacks.ReduceLROnPlateau(
                        monitor='val_accuracy', factor=0.5, patience=10, min_delta=0.01, verbose=0
                    )
                ]
             )

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    print("epochs: "+str(len(acc)))

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print("test loss: {}, test accuracy: {}".format(test_loss, test_acc))

    y_pred = model.predict(X_test)
    print("AUC ROC: ",roc_auc_score(y_test, y_pred))

    y_pred = list(map(lambda i: int(i>=0.5), y_pred))
    print("Kappa: ",cohen_kappa_score(y_test, y_pred))

    print(confusion_matrix(y_test, y_pred))

## Arte Francés y Club De Banqueros y Empresarios y Gramma

   tonalcentroidfiles_1  tonalcentroidfiles_2  tonalcentroidfiles_3  \
0              1.585484              0.923309             -0.748807   
1              1.129768              0.963814              0.209096   
2             -0.066076              1.857866              1.921193   
3              0.119831              1.429286              1.472808   
4             -0.123292              0.197415              0.503797   

   tonalcentroidfiles_4  tonalcentroidfiles_5  tonalcentroidfiles_6  
0              1.209820             -1.073924              0.283035  
1             -0.143471              0.115184              0.020905  
2              1.093619              2.089353              1.984310  
3              1.282029              1.096897              2.122871  
4              1.431215              1.715761              0.683611  
(786, 6)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Los parámetros del mejor modelo fueron {'activation': 'relu', 'hidden_layer_sizes': (30, 30), 'learning_rate_init': 0.004, 'max_iter': 20}, que permiten obtener un Accuracy de 73.34% y un Kappa del 7.33
Model: "model_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 6)                 0         
_________________________________________________________________
dense_58 (Dense)             (None, 30)                210       
_________________________________________________________________
dense_59 (Dense)             (None, 30)                930       
_________________________________________________________________
dense_60 (Dense)             (None, 1)                 31        
Total params: 1,171
Trainable params: 1,171
Non-trainable params: 0
_________________________________________________________________
None
epochs: 20
197/197 [==========================

## Arte Francés y Club De Banqueros y Empresarios y Hotel Marrakech

...

   tonalcentroidfiles_1  tonalcentroidfiles_2  tonalcentroidfiles_3  \
0              0.618349              0.564005              0.091611   
1             -0.149103             -1.303101             -0.498811   
2             -1.141294             -1.317570              0.575363   
3             -0.988346             -0.540855              1.006800   
4             -0.640925             -0.228256              0.461986   

   tonalcentroidfiles_4  tonalcentroidfiles_5  tonalcentroidfiles_6  
0              1.885208              0.563756              1.803892  
1              0.416525             -0.660751             -1.179350  
2             -1.560836              0.336340              1.197709  
3             -0.214650             -0.364940              0.068869  
4             -1.274446             -1.494581             -0.125833  
(651, 6)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (75) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Los parámetros del mejor modelo fueron {'activation': 'relu', 'hidden_layer_sizes': (10, 10), 'learning_rate_init': 0.002, 'max_iter': 75}, que permiten obtener un Accuracy de 65.37% y un Kappa del 7.82
Model: "model_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_39 (InputLayer)        (None, 6)                 0         
_________________________________________________________________
dense_115 (Dense)            (None, 10)                70        
_________________________________________________________________
dense_116 (Dense)            (None, 10)                110       
_________________________________________________________________
dense_117 (Dense)            (None, 1)                 11        
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________
None
epochs: 75
163/163 [==============================

## Hotel Marrakech y Specialized y Urban Place

   tonalcentroidfiles_1  tonalcentroidfiles_2  tonalcentroidfiles_3  \
0              1.297447              0.231356              0.546295   
1              0.766614             -1.227377             -0.605576   
2             -0.431734             -1.183099              1.287522   
3              0.078607             -1.572049              1.188424   
4              0.007655             -1.466562              1.294323   

   tonalcentroidfiles_4  tonalcentroidfiles_5  tonalcentroidfiles_6  
0              1.007345              0.269632              0.236051  
1             -1.216250              0.473081             -0.446139  
2              0.927452             -0.309857             -1.666006  
3              0.092330             -0.244414             -0.490911  
4             -0.410616             -0.763440             -1.007284  
(579, 6)
Los parámetros del mejor modelo fueron {'activation': 'relu', 'hidden_layer_sizes': (20, 20, 20), 'learning_rate_init': 0.003, 'max_iter': 400},

# Chromagram

In [23]:
df_n_ps_std[0].columns

Index(['durationfiles', 'rmsfiles', 'rmsmedianfiles', 'lowenergyfiles',
       'ASRfiles', 'beatspectrumfiles', 'eventdensityfiles', 'tempofiles',
       'pulseclarityfiles', 'zerocrossfiles', 'rolloffsfiles',
       'brightnessfiles', 'spreadfiles', 'centroidfiles', 'kurtosisfiles',
       'flatnessfiles', 'entropyfiles', 'mfccfiles_1', 'mfccfiles_2',
       'mfccfiles_3', 'mfccfiles_4', 'mfccfiles_5', 'mfccfiles_6',
       'mfccfiles_7', 'mfccfiles_8', 'mfccfiles_9', 'mfccfiles_10',
       'mfccfiles_11', 'mfccfiles_12', 'mfccfiles_13', 'inharmonicityfiles',
       'bestkeyfiles', 'keyclarityfiles', 'modalityfiles',
       'tonalcentroidfiles_1', 'tonalcentroidfiles_2', 'tonalcentroidfiles_3',
       'tonalcentroidfiles_4', 'tonalcentroidfiles_5', 'tonalcentroidfiles_6',
       'chromagramfiles_1', 'chromagramfiles_2', 'chromagramfiles_3',
       'chromagramfiles_4', 'chromagramfiles_5', 'chromagramfiles_6',
       'chromagramfiles_7', 'chromagramfiles_8', 'chromagramfiles_9',
      

In [24]:
df_n_ps_std[0].columns[40:52]

Index(['chromagramfiles_1', 'chromagramfiles_2', 'chromagramfiles_3',
       'chromagramfiles_4', 'chromagramfiles_5', 'chromagramfiles_6',
       'chromagramfiles_7', 'chromagramfiles_8', 'chromagramfiles_9',
       'chromagramfiles_10', 'chromagramfiles_11', 'chromagramfiles_12'],
      dtype='object')

In [25]:
df_n_ps_std_ch = [None]*len(companies)
for i in range(len(companies)):
    df_n_ps_std_ch[i] = pd.DataFrame(df_n_ps_std[i].iloc[:,40:52])
    df_n_ps_std_ch[i].columns=df_n_ps_std[i].columns[40:52]
df_n_ps_std_ch[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 12 columns):
chromagramfiles_1     315 non-null float64
chromagramfiles_2     315 non-null float64
chromagramfiles_3     315 non-null float64
chromagramfiles_4     315 non-null float64
chromagramfiles_5     315 non-null float64
chromagramfiles_6     315 non-null float64
chromagramfiles_7     315 non-null float64
chromagramfiles_8     315 non-null float64
chromagramfiles_9     315 non-null float64
chromagramfiles_10    315 non-null float64
chromagramfiles_11    315 non-null float64
chromagramfiles_12    315 non-null float64
dtypes: float64(12)
memory usage: 29.6 KB


## 2-comp groups combinations

In [26]:
from IPython.display import display, Markdown, Latex
import itertools
two_combs = list(itertools.combinations(range(len(companies)), 2))
for comb in two_combs:
    display(Markdown('## '+companies[comb[0]]+' y '+companies[comb[1]]))
    X = df_n_ps_std_ch[comb[0]].append(df_n_ps_std_ch[comb[1]])
    y = df_n_ps[comb[0]]['chosen'].append(df_n_ps[comb[1]]['chosen'])

    print(X.head())
    print(X.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))

    activation_vec = ['logistic', 'relu', 'tanh']
    max_iter_vec = [10, 20, 50, 75, 100, 200, 300, 400, 500, 1000, 2000]
    hidden_layer_sizes_vec = [(10,), (20,), (30,), (10, 10), (20, 20), (30, 30), (20, 10), 
                              (10, 10, 10), (20, 20, 20), (30, 30, 30), (30, 20, 10)]
    learning_rate_init_vec = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02]
    batch_size_vec = [10, 20, 40, 60, 80, 100, 150]


    np.random.seed(1234)
    parametros = {'activation': activation_vec,
                  'max_iter':max_iter_vec,
                  'hidden_layer_sizes': hidden_layer_sizes_vec,
                  'learning_rate_init': learning_rate_init_vec#,
                  #'batch_size': batch_size_vec
                  }
    scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
    grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1, iid=True)

    grid.fit(X_train, y_train)

    print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}".format(
        grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))

    n0=X_train.shape[1]
    ### hidden_layer_sizes
    ns = []
    for i in range (len(grid.best_params_['hidden_layer_sizes'])):
        ns.append(grid.best_params_['hidden_layer_sizes'][i])

    ns.append(1)
    lr = grid.best_params_['learning_rate_init']
    epochs = grid.best_params_['max_iter']

    input_tensor = Input(shape = (n0,))
    hidden_outputs = [input_tensor]
    if grid.best_params_['activation'] == 'logistic':
        grid.best_params_['activation'] = 'sigmoid'
    for i in range (len(ns)-1):
        hidden_outputs.append(Dense(ns[i], activation = grid.best_params_['activation'])(hidden_outputs[i]))
    classification_output = Dense(ns[-1], activation = 'sigmoid')(hidden_outputs[-1])

    model = Model([input_tensor], [classification_output])
    weights = model.get_weights()

    print(model.summary())

    model.set_weights(weights)
    adam = keras.optimizers.Adam(lr=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=0, 
                callbacks=[
                    keras.callbacks.ReduceLROnPlateau(
                        monitor='val_accuracy', factor=0.5, patience=10, min_delta=0.01, verbose=0
                    )
                ]
             )

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    print("epochs: "+str(len(acc)))

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print("test loss: {}, test accuracy: {}".format(test_loss, test_acc))

    y_pred = model.predict(X_test)
    print("AUC ROC: ",roc_auc_score(y_test, y_pred))

    y_pred = list(map(lambda i: int(i>=0.5), y_pred))
    print("Kappa: ",cohen_kappa_score(y_test, y_pred))

    print(confusion_matrix(y_test, y_pred))

## Arte Francés y Club De Banqueros y Empresarios

   chromagramfiles_1  chromagramfiles_2  chromagramfiles_3  chromagramfiles_4  \
0           1.713160           0.038243           2.143431           0.466379   
1           1.315247           0.387677           1.394548          -0.542923   
2           1.737505          -1.440944          -1.114255           0.326790   
3           1.737506          -1.127088          -0.672464           0.505231   
4           1.737506           1.455410           0.333584           1.122628   

   chromagramfiles_5  chromagramfiles_6  chromagramfiles_7  chromagramfiles_8  \
0           2.480688           1.533552          -0.564788          -0.236267   
1          -0.194326           0.100084           0.191568           1.691930   
2          -1.157732          -1.261521          -1.130503           1.585017   
3          -0.830959          -1.046716          -0.912922           1.082754   
4           1.139134          -0.481280           0.129896           0.915597   

   chromagramfiles_9  chro

## Arte Francés y Gramma

   chromagramfiles_1  chromagramfiles_2  chromagramfiles_3  chromagramfiles_4  \
0           1.713160           0.038243           2.143431           0.466379   
1           1.315247           0.387677           1.394548          -0.542923   
2           1.737505          -1.440944          -1.114255           0.326790   
3           1.737506          -1.127088          -0.672464           0.505231   
4           1.737506           1.455410           0.333584           1.122628   

   chromagramfiles_5  chromagramfiles_6  chromagramfiles_7  chromagramfiles_8  \
0           2.480688           1.533552          -0.564788          -0.236267   
1          -0.194326           0.100084           0.191568           1.691930   
2          -1.157732          -1.261521          -1.130503           1.585017   
3          -0.830959          -1.046716          -0.912922           1.082754   
4           1.139134          -0.481280           0.129896           0.915597   

   chromagramfiles_9  chro

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

   chromagramfiles_1  chromagramfiles_2  chromagramfiles_3  chromagramfiles_4  \
0          -0.580880          -0.990276           0.508438           0.658218   
1          -0.886270           1.680400           1.708314           0.223308   
2          -0.133384           0.811454          -1.227033          -0.981611   
3          -0.551071           0.259668          -1.073758          -0.208557   
4          -1.234016          -0.118636          -1.227033          -0.725200   

   chromagramfiles_5  chromagramfiles_6  chromagramfiles_7  chromagramfiles_8  \
0           1.668751          -0.289029          -0.581158           1.476860   
1           0.928680          -0.795001           0.621735          -0.834425   
2          -1.054713          -1.027505          -0.745868           0.198884   
3           0.002105          -0.655399          -0.056661          -1.206025   
4          -1.112673          -0.992824          -0.809889          -1.428072   

   chromagramfiles_9  chro

## Specialized y Urban Place

   chromagramfiles_1  chromagramfiles_2  chromagramfiles_3  chromagramfiles_4  \
0           1.752761          -1.114598          -0.348132           2.966304   
1          -1.387006          -0.333039           0.041297          -0.917052   
2          -0.628834           2.234144           0.613536          -0.978644   
3           0.081693           1.765530          -0.365668           0.759057   
4           0.056206           1.646501           0.508800           0.525847   

   chromagramfiles_5  chromagramfiles_6  chromagramfiles_7  chromagramfiles_8  \
0          -0.925235           0.552064          -0.710296           0.412009   
1           0.656635          -1.022407          -0.805166          -0.905135   
2           0.836157          -0.735689          -0.059767          -1.571350   
3          -1.136519          -0.071939          -0.412587          -1.310708   
4           0.506842          -0.390517          -0.241209          -0.409725   

   chromagramfiles_9  chro

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (75) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Los parámetros del mejor modelo fueron {'activation': 'relu', 'hidden_layer_sizes': (30, 30, 30), 'learning_rate_init': 0.005, 'max_iter': 75}, que permiten obtener un Accuracy de 69.94% y un Kappa del 37.58
Model: "model_55"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_55 (InputLayer)        (None, 12)                0         
_________________________________________________________________
dense_163 (Dense)            (None, 30)                390       
_________________________________________________________________
dense_164 (Dense)            (None, 30)                930       
_________________________________________________________________
dense_165 (Dense)            (None, 30)                930       
_________________________________________________________________
dense_166 (Dense)            (None, 1)                 31        
Total params: 2,281
Trainable params: 2,281
Non-

## 3-comp groups combinations

In [27]:
from IPython.display import display, Markdown, Latex
import itertools
three_combs = list(itertools.combinations(range(len(companies)), 3))
for comb in three_combs:
    title = '## '
    for no in comb[:-1]:
        title+=companies[no]
        title+=' y '
    title+=companies[comb[-1]]
    display(Markdown(title))
    X = df_n_ps_std_ch[comb[0]]
    y = df_n_ps[comb[0]]['chosen']
    for no in comb[1:]:
        X= X.append(df_n_ps_std_ch[no])
        y= y.append(df_n_ps[no]['chosen'])
    

    print(X.head())
    print(X.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))

    activation_vec = ['logistic', 'relu', 'tanh']
    max_iter_vec = [10, 20, 50, 75, 100, 200, 300, 400, 500, 1000, 2000]
    hidden_layer_sizes_vec = [(10,), (20,), (30,), (10, 10), (20, 20), (30, 30), (20, 10), 
                              (10, 10, 10), (20, 20, 20), (30, 30, 30), (30, 20, 10)]
    learning_rate_init_vec = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02]
    batch_size_vec = [10, 20, 40, 60, 80, 100, 150]


    np.random.seed(1234)
    parametros = {'activation': activation_vec,
                  'max_iter':max_iter_vec,
                  'hidden_layer_sizes': hidden_layer_sizes_vec,
                  'learning_rate_init': learning_rate_init_vec#,
                  #'batch_size': batch_size_vec
                  }
    scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
    grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1, iid=True)

    grid.fit(X_train, y_train)

    print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}".format(
        grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))

    n0=X_train.shape[1]
    ### hidden_layer_sizes
    ns = []
    for i in range (len(grid.best_params_['hidden_layer_sizes'])):
        ns.append(grid.best_params_['hidden_layer_sizes'][i])

    ns.append(1)
    lr = grid.best_params_['learning_rate_init']
    epochs = grid.best_params_['max_iter']

    input_tensor = Input(shape = (n0,))
    hidden_outputs = [input_tensor]
    if grid.best_params_['activation'] == 'logistic':
        grid.best_params_['activation'] = 'sigmoid'
    for i in range (len(ns)-1):
        hidden_outputs.append(Dense(ns[i], activation = grid.best_params_['activation'])(hidden_outputs[i]))
    classification_output = Dense(ns[-1], activation = 'sigmoid')(hidden_outputs[-1])

    model = Model([input_tensor], [classification_output])
    weights = model.get_weights()

    print(model.summary())

    model.set_weights(weights)
    adam = keras.optimizers.Adam(lr=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=0, 
                callbacks=[
                    keras.callbacks.ReduceLROnPlateau(
                        monitor='val_accuracy', factor=0.5, patience=10, min_delta=0.01, verbose=0
                    )
                ]
             )

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    print("epochs: "+str(len(acc)))

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print("test loss: {}, test accuracy: {}".format(test_loss, test_acc))

    y_pred = model.predict(X_test)
    print("AUC ROC: ",roc_auc_score(y_test, y_pred))

    y_pred = list(map(lambda i: int(i>=0.5), y_pred))
    print("Kappa: ",cohen_kappa_score(y_test, y_pred))

    print(confusion_matrix(y_test, y_pred))

## Arte Francés y Club De Banqueros y Empresarios y Gramma

   chromagramfiles_1  chromagramfiles_2  chromagramfiles_3  chromagramfiles_4  \
0           1.713160           0.038243           2.143431           0.466379   
1           1.315247           0.387677           1.394548          -0.542923   
2           1.737505          -1.440944          -1.114255           0.326790   
3           1.737506          -1.127088          -0.672464           0.505231   
4           1.737506           1.455410           0.333584           1.122628   

   chromagramfiles_5  chromagramfiles_6  chromagramfiles_7  chromagramfiles_8  \
0           2.480688           1.533552          -0.564788          -0.236267   
1          -0.194326           0.100084           0.191568           1.691930   
2          -1.157732          -1.261521          -1.130503           1.585017   
3          -0.830959          -1.046716          -0.912922           1.082754   
4           1.139134          -0.481280           0.129896           0.915597   

   chromagramfiles_9  chro

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Los parámetros del mejor modelo fueron {'activation': 'relu', 'hidden_layer_sizes': (30, 30), 'learning_rate_init': 0.01, 'max_iter': 10}, que permiten obtener un Accuracy de 72.50% y un Kappa del 3.55
Model: "model_56"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_56 (InputLayer)        (None, 12)                0         
_________________________________________________________________
dense_167 (Dense)            (None, 30)                390       
_________________________________________________________________
dense_168 (Dense)            (None, 30)                930       
_________________________________________________________________
dense_169 (Dense)            (None, 1)                 31        
Total params: 1,351
Trainable params: 1,351
Non-trainable params: 0
_________________________________________________________________
None
epochs: 10
197/197 [===========================

## Arte Francés y Club De Banqueros y Empresarios y Hotel Marrakech

...

   chromagramfiles_1  chromagramfiles_2  chromagramfiles_3  chromagramfiles_4  \
0           2.134448          -0.644769          -0.741425          -0.871269   
1          -0.027862           1.690489           0.055411          -0.561004   
2          -0.377201           2.125071          -0.304151           2.056775   
3          -0.590231           0.143657          -0.601072           1.282246   
4          -0.233782           0.378000           1.366350           0.941326   

   chromagramfiles_5  chromagramfiles_6  chromagramfiles_7  chromagramfiles_8  \
0           0.189298          -0.623423          -0.529336           0.357204   
1           0.495091          -0.306711           0.886175          -0.561042   
2          -0.998633          -0.428416           1.821160          -1.229826   
3          -0.578812          -0.045810           1.094766           0.365693   
4          -0.898589          -0.324641           1.117103           0.022453   

   chromagramfiles_9  chro

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Los parámetros del mejor modelo fueron {'activation': 'tanh', 'hidden_layer_sizes': (10, 10), 'learning_rate_init': 0.01, 'max_iter': 50}, que permiten obtener un Accuracy de 67.42% y un Kappa del 25.69
Model: "model_74"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_74 (InputLayer)        (None, 12)                0         
_________________________________________________________________
dense_220 (Dense)            (None, 10)                130       
_________________________________________________________________
dense_221 (Dense)            (None, 10)                110       
_________________________________________________________________
dense_222 (Dense)            (None, 1)                 11        
Total params: 251
Trainable params: 251
Non-trainable params: 0
_________________________________________________________________
None
epochs: 50
163/163 [==============================

## Hotel Marrakech y Specialized y Urban Place

   chromagramfiles_1  chromagramfiles_2  chromagramfiles_3  chromagramfiles_4  \
0          -0.580880          -0.990276           0.508438           0.658218   
1          -0.886270           1.680400           1.708314           0.223308   
2          -0.133384           0.811454          -1.227033          -0.981611   
3          -0.551071           0.259668          -1.073758          -0.208557   
4          -1.234016          -0.118636          -1.227033          -0.725200   

   chromagramfiles_5  chromagramfiles_6  chromagramfiles_7  chromagramfiles_8  \
0           1.668751          -0.289029          -0.581158           1.476860   
1           0.928680          -0.795001           0.621735          -0.834425   
2          -1.054713          -1.027505          -0.745868           0.198884   
3           0.002105          -0.655399          -0.056661          -1.206025   
4          -1.112673          -0.992824          -0.809889          -1.428072   

   chromagramfiles_9  chro

## 4-comp groups combinations

In [29]:
from IPython.display import display, Markdown, Latex
import itertools
three_combs = list(itertools.combinations(range(len(companies)), 4))
for comb in three_combs:
    title = '## '
    for no in comb[:-1]:
        title+=companies[no]
        title+=' y '
    title+=companies[comb[-1]]
    display(Markdown(title))
    X = df_n_ps_std_ch[comb[0]]
    y = df_n_ps[comb[0]]['chosen']
    for no in comb[1:]:
        X= X.append(df_n_ps_std_ch[no])
        y= y.append(df_n_ps[no]['chosen'])
    

    print(X.head())
    print(X.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))

    activation_vec = ['logistic', 'relu', 'tanh']
    max_iter_vec = [10, 20, 50, 75, 100, 200, 300, 400, 500, 1000, 2000]
    hidden_layer_sizes_vec = [(10,), (20,), (30,), (10, 10), (20, 20), (30, 30), (20, 10), 
                              (10, 10, 10), (20, 20, 20), (30, 30, 30), (30, 20, 10)]
    learning_rate_init_vec = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02]
    batch_size_vec = [10, 20, 40, 60, 80, 100, 150]


    np.random.seed(1234)
    parametros = {'activation': activation_vec,
                  'max_iter':max_iter_vec,
                  'hidden_layer_sizes': hidden_layer_sizes_vec,
                  'learning_rate_init': learning_rate_init_vec#,
                  #'batch_size': batch_size_vec
                  }
    scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
    grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1, iid=True)

    grid.fit(X_train, y_train)

    print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}".format(
        grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))

    n0=X_train.shape[1]
    ### hidden_layer_sizes
    ns = []
    for i in range (len(grid.best_params_['hidden_layer_sizes'])):
        ns.append(grid.best_params_['hidden_layer_sizes'][i])

    ns.append(1)
    lr = grid.best_params_['learning_rate_init']
    epochs = grid.best_params_['max_iter']

    input_tensor = Input(shape = (n0,))
    hidden_outputs = [input_tensor]
    if grid.best_params_['activation'] == 'logistic':
        grid.best_params_['activation'] = 'sigmoid'
    for i in range (len(ns)-1):
        hidden_outputs.append(Dense(ns[i], activation = grid.best_params_['activation'])(hidden_outputs[i]))
    classification_output = Dense(ns[-1], activation = 'sigmoid')(hidden_outputs[-1])

    model = Model([input_tensor], [classification_output])
    weights = model.get_weights()

    print(model.summary())

    model.set_weights(weights)
    adam = keras.optimizers.Adam(lr=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=0, 
                callbacks=[
                    keras.callbacks.ReduceLROnPlateau(
                        monitor='val_accuracy', factor=0.5, patience=10, min_delta=0.01, verbose=0
                    )
                ]
             )

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    print("epochs: "+str(len(acc)))

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print("test loss: {}, test accuracy: {}".format(test_loss, test_acc))

    y_pred = model.predict(X_test)
    print("AUC ROC: ",roc_auc_score(y_test, y_pred))

    y_pred = list(map(lambda i: int(i>=0.5), y_pred))
    print("Kappa: ",cohen_kappa_score(y_test, y_pred))

    print(confusion_matrix(y_test, y_pred))

## Arte Francés y Club De Banqueros y Empresarios y Gramma y Hotel Marrakech

   chromagramfiles_1  chromagramfiles_2  chromagramfiles_3  chromagramfiles_4  \
0           1.713160           0.038243           2.143431           0.466379   
1           1.315247           0.387677           1.394548          -0.542923   
2           1.737505          -1.440944          -1.114255           0.326790   
3           1.737506          -1.127088          -0.672464           0.505231   
4           1.737506           1.455410           0.333584           1.122628   

   chromagramfiles_5  chromagramfiles_6  chromagramfiles_7  chromagramfiles_8  \
0           2.480688           1.533552          -0.564788          -0.236267   
1          -0.194326           0.100084           0.191568           1.691930   
2          -1.157732          -1.261521          -1.130503           1.585017   
3          -0.830959          -1.046716          -0.912922           1.082754   
4           1.139134          -0.481280           0.129896           0.915597   

   chromagramfiles_9  chro

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Los parámetros del mejor modelo fueron {'activation': 'relu', 'hidden_layer_sizes': (10, 10), 'learning_rate_init': 0.007, 'max_iter': 20}, que permiten obtener un Accuracy de 69.58% y un Kappa del 3.22
Model: "model_91"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_91 (InputLayer)        (None, 12)                0         
_________________________________________________________________
dense_276 (Dense)            (None, 10)                130       
_________________________________________________________________
dense_277 (Dense)            (None, 10)                110       
_________________________________________________________________
dense_278 (Dense)            (None, 1)                 11        
Total params: 251
Trainable params: 251
Non-trainable params: 0
_________________________________________________________________
None
epochs: 20
233/233 [==============================

## Arte Francés y Club De Banqueros y Empresarios y Gramma y Specialized

...

   chromagramfiles_1  chromagramfiles_2  chromagramfiles_3  chromagramfiles_4  \
0          -0.369691          -0.881824          -0.095656          -0.923999   
1          -0.175875          -0.403800          -0.657709          -0.201259   
2           0.894452          -0.189794           1.959063           0.169276   
3           0.060782          -0.392075           0.826233          -0.048480   
4          -1.116536          -0.923742          -1.238971          -0.919598   

   chromagramfiles_5  chromagramfiles_6  chromagramfiles_7  chromagramfiles_8  \
0          -0.030645          -0.834931          -1.031650          -0.840942   
1           1.691433          -0.672783          -0.119944          -0.440080   
2          -0.403611          -1.036954           1.447615          -0.340767   
3           1.789786          -0.552163           0.121028          -0.111355   
4           0.313068          -1.160111           1.316032          -0.700013   

   chromagramfiles_9  chro

## Gramma y Hotel Marrakech y Specialized y Urban Place

   chromagramfiles_1  chromagramfiles_2  chromagramfiles_3  chromagramfiles_4  \
0           2.134448          -0.644769          -0.741425          -0.871269   
1          -0.027862           1.690489           0.055411          -0.561004   
2          -0.377201           2.125071          -0.304151           2.056775   
3          -0.590231           0.143657          -0.601072           1.282246   
4          -0.233782           0.378000           1.366350           0.941326   

   chromagramfiles_5  chromagramfiles_6  chromagramfiles_7  chromagramfiles_8  \
0           0.189298          -0.623423          -0.529336           0.357204   
1           0.495091          -0.306711           0.886175          -0.561042   
2          -0.998633          -0.428416           1.821160          -1.229826   
3          -0.578812          -0.045810           1.094766           0.365693   
4          -0.898589          -0.324641           1.117103           0.022453   

   chromagramfiles_9  chro

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Los parámetros del mejor modelo fueron {'activation': 'logistic', 'hidden_layer_sizes': (10,), 'learning_rate_init': 0.003, 'max_iter': 10}, que permiten obtener un Accuracy de 63.26% y un Kappa del 2.21
Model: "model_105"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_105 (InputLayer)       (None, 12)                0         
_________________________________________________________________
dense_318 (Dense)            (None, 10)                130       
_________________________________________________________________
dense_319 (Dense)            (None, 1)                 11        
Total params: 141
Trainable params: 141
Non-trainable params: 0
_________________________________________________________________
None
epochs: 10
199/199 [==============================] - 0s 55us/step
test loss: 0.6593509022913986, test accuracy: 0.6180904507637024
AUC ROC:  0.5637641047477113
Kappa:  0.015877147

## 5-comp groups combinations

In [ ]:
from IPython.display import display, Markdown, Latex
import itertools
three_combs = list(itertools.combinations(range(len(companies)), 5))
for comb in three_combs:
    title = '## '
    for no in comb[:-1]:
        title+=companies[no]
        title+=' y '
    title+=companies[comb[-1]]
    display(Markdown(title))
    X = df_n_ps_std_ch[comb[0]]
    y = df_n_ps[comb[0]]['chosen']
    for no in comb[1:]:
        X= X.append(df_n_ps_std_ch[no])
        y= y.append(df_n_ps[no]['chosen'])
    

    print(X.head())
    print(X.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))

    activation_vec = ['logistic', 'relu', 'tanh']
    max_iter_vec = [10, 20, 50, 75, 100, 200, 300, 400, 500, 1000, 2000]
    hidden_layer_sizes_vec = [(10,), (20,), (30,), (10, 10), (20, 20), (30, 30), (20, 10), 
                              (10, 10, 10), (20, 20, 20), (30, 30, 30), (30, 20, 10)]
    learning_rate_init_vec = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02]
    batch_size_vec = [10, 20, 40, 60, 80, 100, 150]


    np.random.seed(1234)
    parametros = {'activation': activation_vec,
                  'max_iter':max_iter_vec,
                  'hidden_layer_sizes': hidden_layer_sizes_vec,
                  'learning_rate_init': learning_rate_init_vec#,
                  #'batch_size': batch_size_vec
                  }
    scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
    grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1, iid=True)

    grid.fit(X_train, y_train)

    print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}".format(
        grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))

    n0=X_train.shape[1]
    ### hidden_layer_sizes
    ns = []
    for i in range (len(grid.best_params_['hidden_layer_sizes'])):
        ns.append(grid.best_params_['hidden_layer_sizes'][i])

    ns.append(1)
    lr = grid.best_params_['learning_rate_init']
    epochs = grid.best_params_['max_iter']

    input_tensor = Input(shape = (n0,))
    hidden_outputs = [input_tensor]
    if grid.best_params_['activation'] == 'logistic':
        grid.best_params_['activation'] = 'sigmoid'
    for i in range (len(ns)-1):
        hidden_outputs.append(Dense(ns[i], activation = grid.best_params_['activation'])(hidden_outputs[i]))
    classification_output = Dense(ns[-1], activation = 'sigmoid')(hidden_outputs[-1])

    model = Model([input_tensor], [classification_output])
    weights = model.get_weights()

    print(model.summary())

    model.set_weights(weights)
    adam = keras.optimizers.Adam(lr=lr)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=0, 
                callbacks=[
                    keras.callbacks.ReduceLROnPlateau(
                        monitor='val_accuracy', factor=0.5, patience=10, min_delta=0.01, verbose=0
                    )
                ]
             )

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    print("epochs: "+str(len(acc)))

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print("test loss: {}, test accuracy: {}".format(test_loss, test_acc))

    y_pred = model.predict(X_test)
    print("AUC ROC: ",roc_auc_score(y_test, y_pred))

    y_pred = list(map(lambda i: int(i>=0.5), y_pred))
    print("Kappa: ",cohen_kappa_score(y_test, y_pred))

    print(confusion_matrix(y_test, y_pred))

## Arte Francés y Club De Banqueros y Empresarios y Gramma y Hotel Marrakech y Specialized

   chromagramfiles_1  chromagramfiles_2  chromagramfiles_3  chromagramfiles_4  \
0           1.713160           0.038243           2.143431           0.466379   
1           1.315247           0.387677           1.394548          -0.542923   
2           1.737505          -1.440944          -1.114255           0.326790   
3           1.737506          -1.127088          -0.672464           0.505231   
4           1.737506           1.455410           0.333584           1.122628   

   chromagramfiles_5  chromagramfiles_6  chromagramfiles_7  chromagramfiles_8  \
0           2.480688           1.533552          -0.564788          -0.236267   
1          -0.194326           0.100084           0.191568           1.691930   
2          -1.157732          -1.261521          -1.130503           1.585017   
3          -0.830959          -1.046716          -0.912922           1.082754   
4           1.139134          -0.481280           0.129896           0.915597   

   chromagramfiles_9  chro